In [ ]:
!pip install geopandas

In [ ]:
from tqdm import tqdm
from shapely.geometry import shape
import geopandas as gpd
import os
import json
import tqdm
%matplotlib inline 

In [ ]:
state           = "arkansas"
working_dir     = "C:\\Users\\pjrio\\tl_2020_05_bg"
if not os.path.isdir(working_dir):
    print("Directory is not valid.")

In [ ]:
# paths to files
counties_file_path  = os.path.join(f"{working_dir}",f"{state}-counties.geojson")
nra_file_path       = os.path.join(f"{working_dir}",f"{state}-nra.geojson")
buildings_file_path = os.path.join(f"{working_dir}",f"{state}.geojson")

# Link to the block group level files https://gis.arkansas.gov/product/block-groups-2020-census/
group_data_path     = os.path.join(f"{working_dir}","shape2geojson.geojson")

# paths to directories
osm_counties_folder = os.path.join(f"{working_dir}","osm_counties")
ms_counties_folder  = os.path.join(f"{working_dir}","ms_counties")
counties_filtered   = os.path.join(f"{working_dir}","counties_filtered")

files_to_check      = [counties_file_path, nra_file_path, buildings_file_path]

# check if files exist, print message if not
for file_path in files_to_check:
    if not os.path.isfile(file_path):
        print("File {} does not exist".format(file_path))

# Create output directory if it doesn't exist
os.makedirs(osm_counties_folder, exist_ok=True)
os.makedirs(ms_counties_folder , exist_ok=True)
os.makedirs(counties_filtered  , exist_ok=True)

print("Paths to files:")
print("--counties_file_path : " + counties_file_path)
print("--nra_file_path      : " + nra_file_path)
print("--buildings_file_path: " + buildings_file_path + "\n")

print("Paths to directories:")
print("--OSM counties folder: " + osm_counties_folder)
print("--MS counties folder : " + ms_counties_folder)
print("--Filtered counties  : " + counties_filtered)

In [ ]:
# Load counties boundaries file
print("Loading boundaries file...")
with open(counties_file_path, 'r') as f:
    boudaries_data = json.load(f)

# Convert the county data to a GeoDataFrame
counties = gpd.GeoDataFrame.from_features(boudaries_data["features"])
print("Boundaries file was loaded.")

# Divide by Counties

In [ ]:
def create_county_geojson(buildings_file_path, output_dir):
    # Load the building data
    print("Loading buildings...")
    with open(buildings_file_path) as f:
        building_data = json.load(f)
    buildings = gpd.GeoDataFrame.from_features(building_data)
    
    # Iterate over each county and create a separate GeoJSON file for each
    for i, county in tqdm.tqdm(counties.iterrows(), total=len(counties), desc="Processing counties"):
        # Get the geometry for the current county
        county_geometry = county["geometry"]

        # Get the buildings that fall within the current county
        county_buildings = buildings[buildings.within(county_geometry)]

        # Save the buildings for the current county to a new GeoJSON file
        county_name = county["name"]
        state_name = county["state_name"]
        county_outputdir =  os.path.join(f"{output_dir}",f"{state_name}_{county_name}.geojson")
        county_buildings.to_file(county_outputdir, driver="GeoJSON")

### Split MS into counties
Given the geojson file of a state from the MS dataset and the conty boundaries of that state, split the state file into smaller states based on the conty divisions.

In [ ]:
#Split MS data into counties
print("Split MS data into counties")
create_county_geojson(buildings_file_path, ms_counties_folder)

### Split OSM into counties
Given the geojson file of a state from the OSM dataset and the conty boundaries of that state, split the state file into smaller states based on the conty divisions.

In [ ]:
#Split MS data into counties
print("Split OSM data into counties")
create_county_geojson(nra_file_path, osm_counties_folder)

# Filter

In [ ]:
# Loop through all the GeoJSON files in the ms_counties folder
for filename in os.listdir(ms_counties_folder):
    print(filename)
    if filename.endswith('.geojson'):
        ms_filepath = os.path.join(ms_counties_folder, filename)
        osm_filepath = os.path.join(osm_counties_folder, filename)
        output_filepath = os.path.join(counties_filtered, 'filtered_' + filename)

        #print(output_filepath)
        if os.path.exists(output_filepath):
            # skip the file
            print("--"+filename +" has already been filtered")
            continue

        # Load the GeoJSON data for both the ms_counties and osm_counties files
        with open(ms_filepath, 'r') as f:
            ms_data = json.load(f)
        with open(osm_filepath, 'r') as f:
            osm_data = json.load(f)

        # Create a Shapely object for each feature in the osm_counties file
        osm_features = []
        for feature in osm_data['features']:
            osm_features.append(shape(feature['geometry']))

        #print("Porcessing: " + filename)
        # Filter the features in the ms_counties file
        filtered_features = []
        with tqdm.tqdm(total=len(ms_data['features'])) as pbar:
            for feature in ms_data['features']:
                feature_polygon = shape(feature['geometry'])
                for osm_feature in osm_features:
                    if osm_feature.contains(feature_polygon):
                        break
                else:
                    filtered_features.append(feature)
                pbar.update(1)

        # Write the filtered features to a new GeoJSON file
        ms_data['features'] = filtered_features
        with open(output_filepath, 'w') as f:
            json.dump(ms_data, f)

# Split into block groups

Load the FIP state information by county and make a dictionary
https://www.census.gov/library/reference/code-lists/ansi.html#county

In [ ]:
# Load the county name data
county_name_path = f"C:\\Users\\pjrio\\tl_2020_05_bg\\merge\\{state}.txt"
county_names = {}
with open(county_name_path) as f:
    for line in f:
        #print(line)
        county_ab, state_number, county_number, county_name, something = line.strip().split(",")
        county_names[county_number] = county_name
#print(len(county_names))
print(county_names)

## Load the group level boundary data

In [ ]:
# Load the group level boundary data
print("Loading group level boundaries...")
with open(group_data_path) as f:
    group_data = json.load(f)

# Convert the group level boundary data to a GeoDataFrame
groups = gpd.GeoDataFrame.from_features(group_data["features"])

In [ ]:
def create_group_geojson(buildings_file_path, group_file_path, output_dir):
    basename = os.path.basename(buildings_file_path).replace(".geojson","")
    basename = basename.lower()
    basename = basename.replace(f"filtered_{state}_","")
    
    print(basename)
    # Load the building data
    print("Loading buildings...")
    with open(buildings_file_path) as f:
        building_data = json.load(f)
    buildings = gpd.GeoDataFrame.from_features(building_data)
    
    # Iterate over each group and create a separate GeoJSON file for each
    for i, group in tqdm.tqdm(groups.iterrows(), total=len(groups), desc="Processing groups"):
        county_number = group["COUNTYFP"]
        county_name = county_names[county_number].replace("County","")  # look up county name in dictionary
        county_name = county_name.replace(" ","")  # look up county name in dictionary
        county_name = county_name.lower()
        #print(county_name.lower())
        if(county_name in basename.lower()):
            #print(county_name)
            #Get the geometry for the current group
            group_geometry = group["geometry"]

            # Get the buildings that fall within the current group
            group_buildings = buildings[buildings.within(group_geometry)]

            # Save the buildings for the current group to a new GeoJSON file
            state_name = state
            county_number = group["COUNTYFP"]
            county_name = county_names[county_number]  # look up county name in dictionary
            tract_name = group["TRACTCE"]
            group_name = group["BLKGRPCE"]
            output_path = os.path.join(output_dir, county_name, tract_name, f"block_{group_name}.geojson")
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            group_buildings.to_file(output_path, driver="GeoJSON")

In [ ]:
subdirectories   = [name for name in os.listdir(counties_filtered)]
#print(subdirectories)
for subdirectory in subdirectories:  
    buildings_file_path = counties_filtered +f"\\{subdirectory}"
    #print(buildings_file_path)
    create_group_geojson(buildings_file_path, group_data_path, counties_filtered)